# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("cities.csv")
#added this live to remove Unnamed 0 column
del weather_df["Unnamed: 0"]
weather_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Ushuaia,-54.8000,-68.3000,36.81,89,100,6.58,AR,1627416209
1,Mataura,-46.1927,168.8643,48.47,97,25,1.25,NZ,1627416293
2,Busselton,-33.6500,115.3333,46.04,64,100,12.33,AU,1627415844
3,Cape Town,-33.9258,18.4232,56.08,85,94,4.00,ZA,1627416295
4,Namatanai,-3.6667,152.4333,78.94,82,53,4.70,PG,1627416080
...,...,...,...,...,...,...,...,...,...
525,Puerto Limón,10.0000,-83.0333,82.72,73,48,5.30,CR,1627417019
526,Cukai,4.2500,103.4167,80.55,89,67,3.62,MY,1627417020
527,Rio Grande,-32.0350,-52.0986,50.11,50,6,20.24,BR,1627417021
528,Jesup,31.6074,-81.8854,93.13,52,75,0.00,US,1627416785


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
gmaps.configure(api_key=g_key)
# Store 'Lat' and 'Lng' into  locations 
locations = weather_df[["Lat", "Lng"]]
humidity = weather_df["Humidity"].astype(float)
fig = gmaps.figure()
 
#adding heat map with max intensiy
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
#added humidity to dataset to limit data.
perfect_weather = weather_df.loc[(weather_df["Max Temp"]>=70)&(weather_df["Max Temp"]<=80)&
                                (weather_df["Cloudiness"]==0)&(weather_df["Wind Speed"]<10)].dropna()
                                 #(weather_df["Humidity"]<=60)].dropna()
perfect_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
65,East London,-33.0153,27.9116,73.15,30,0,8.08,ZA,1627416385
77,Darnah,32.7670,22.6367,78.15,69,0,8.97,LY,1627416400
96,Arraial do Cabo,-22.9661,-42.0278,76.86,69,0,6.91,BR,1627416427
112,Alabushevo,56.0100,37.1500,73.40,53,0,6.71,RU,1627416449
134,Costineşti,43.9500,28.6333,77.02,59,0,8.81,RO,1627416477
192,Santa Pola,38.1917,-0.5658,79.97,79,0,2.30,ES,1627416559
293,Marfino,55.7028,37.3822,73.87,62,0,2.24,RU,1627416706
350,Shahrud,36.4182,54.9763,78.35,44,0,6.40,IR,1627416781
368,San Javier,37.8063,-0.8374,79.68,76,0,1.99,ES,1627416806
373,Coswig,51.1333,13.5833,73.98,66,0,2.30,DE,1627416813


In [5]:
#5 to 10 cities max

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
hotel_df = perfect_weather
hotel_df ["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
65,East London,-33.0153,27.9116,73.15,30,0,8.08,ZA,1627416385,
77,Darnah,32.7670,22.6367,78.15,69,0,8.97,LY,1627416400,
96,Arraial do Cabo,-22.9661,-42.0278,76.86,69,0,6.91,BR,1627416427,
112,Alabushevo,56.0100,37.1500,73.40,53,0,6.71,RU,1627416449,
134,Costineşti,43.9500,28.6333,77.02,59,0,8.81,RO,1627416477,
192,Santa Pola,38.1917,-0.5658,79.97,79,0,2.30,ES,1627416559,
293,Marfino,55.7028,37.3822,73.87,62,0,2.24,RU,1627416706,
350,Shahrud,36.4182,54.9763,78.35,44,0,6.40,IR,1627416781,
368,San Javier,37.8063,-0.8374,79.68,76,0,1.99,ES,1627416806,
373,Coswig,51.1333,13.5833,73.98,66,0,2.30,DE,1627416813,


In [14]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get lat and lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City"]
    
    params = {
    "location": f"{lat}, {lng}",  
    "radius": 5000,
    "type": "Hotel",
    "key": g_key,}

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()
   
    results = response['results']
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        print(f"Closest hotel in {city} is {results[0]['name']}.")
        
           
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    
hotel_df

Retrieving Results for Index 65: East London.
Closest hotel in East London is East London.
Retrieving Results for Index 77: Darnah.
Closest hotel in Darnah is Derna.
Retrieving Results for Index 96: Arraial do Cabo.
Closest hotel in Arraial do Cabo is Arraial do Cabo.
Retrieving Results for Index 112: Alabushevo.
Closest hotel in Alabushevo is Moscow.
Retrieving Results for Index 134: Costineşti.
Closest hotel in Costineşti is Costinești.
Retrieving Results for Index 192: Santa Pola.
Closest hotel in Santa Pola is Santa Pola.
Retrieving Results for Index 293: Marfino.
Closest hotel in Marfino is Moscow.
Retrieving Results for Index 350: Shahrud.
Closest hotel in Shahrud is Shahroud.
Retrieving Results for Index 368: San Javier.
Closest hotel in San Javier is San Javier, Murcia.
Retrieving Results for Index 373: Coswig.
Closest hotel in Coswig is Dresden.


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
65,East London,-33.0153,27.9116,73.15,30,0,8.08,ZA,1627416385,East London
77,Darnah,32.7670,22.6367,78.15,69,0,8.97,LY,1627416400,Derna
96,Arraial do Cabo,-22.9661,-42.0278,76.86,69,0,6.91,BR,1627416427,Arraial do Cabo
112,Alabushevo,56.0100,37.1500,73.40,53,0,6.71,RU,1627416449,Moscow
134,Costineşti,43.9500,28.6333,77.02,59,0,8.81,RO,1627416477,Costinești
192,Santa Pola,38.1917,-0.5658,79.97,79,0,2.30,ES,1627416559,Santa Pola
293,Marfino,55.7028,37.3822,73.87,62,0,2.24,RU,1627416706,Moscow
350,Shahrud,36.4182,54.9763,78.35,44,0,6.40,IR,1627416781,Shahroud
368,San Javier,37.8063,-0.8374,79.68,76,0,1.99,ES,1627416806,"San Javier, Murcia"
373,Coswig,51.1333,13.5833,73.98,66,0,2.30,DE,1627416813,Dresden


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# Add marker layer ontop of heat map
# Display figure
markers = gmaps.marker_layer(locations,
    info_box_content= hotel_info)
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))